In [1]:
import sys
sys.path.append('../ry')
#sys.path.append('/home/christina/rai-python/env/lib/python3.5/site-packages')
#sys.path.append('/home/christina/rai-python/env/lib/python3.5/site-packages/IPython/extensions')
#sys.path.append('/home/christina/.ipython')
#print (sys.path)
from libry import *
from numpy import *

In [2]:
K = Config()
K.addFile("../rai-robotModels/kuka_drake/setup.g")
K.addFile("../models/mainipulationStation.g")

K.addFrame("obj0", "tabletop", "type:ssBox size:[.06 .05 .10 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.15 -0.2 0.07)>" )
K.addFrame("obj1", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.3 .2 .15)>" )
K.addFrame("obj2", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.4 .4 .15)>" )
K.addFrame("obj3", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.1 0 .15)>" )
K.addFrame("obj4", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.4 -.15 .15)>" )

#K.addFrame("obj3", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.5 .6.15)>" )
#K.addFrame("tray", "shelfLower", "type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:<t(-0.1 0 .07)>" );
#K.addFrame("", "tray", "type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]" )
q = K.getJointState()
q = [0,0,0,-1.75,0,1,0,0.05]
K.setJointState(q)
V = K.view()

In [3]:
lgp = K.lgp("../models/fol-pickAndPlace.g")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '<ROOT>',
 'path': '',
 'state': ' STATE_0(START_STATE) {(gripper endeff), (table bottom), (table shelfLower), (table shelfUpper), (object obj0), (object obj1), (object obj2), (object obj3), (object obj4), (partOf tabletop upperRightPost), (partOf tabletop upperLeftPost), (partOf tabletop lowerLeftPost), (partOf tabletop lowerRightPost), (partOf tabletop topLeftBar), (partOf tabletop topMiddleBar), (partOf tabletop topRightBar), (partOf tabletop topBackBar), (partOf tabletop topFrontBar), (partOf tabletop rightWall), (partOf tabletop leftWall), (partOf tabletop bottom), (partOf tabletop top), (partOf tabletop shelfLower), (partOf tabletop shelfUpper), (partOf upperRightPost tabletop), (partOf upperRightPost upperLeftPost), (partOf upperRightPost lowerLeftPost), (partOf upperRightPost lowerRightPost), (partOf upperRightPost topLeftBar), (partOf uppe

In [4]:
lgp.getDecisions()

['(grasp endeff obj0)',
 '(grasp endeff obj1)',
 '(grasp endeff obj2)',
 '(grasp endeff obj3)',
 '(grasp endeff obj4)']

In [5]:
import time

lgp.addTerminalRule("(on obj0 bottom) (on obj1 shelfLower) (on obj2 bottom) (on obj3 shelfUpper) (on obj4 bottom)")
lgp.run(2)
while (lgp.numSolutions() < 3):
    time.sleep(1)

lgp.stop()

In [6]:
lgp.numSolutions()

2

In [7]:
lgp.getReport(0, BT.seq)

[{'description': 'qItself:ALL',
  'scale': 0.01,
  'sos_value': 6.863697475950466,
  'type': 'sos'},
 {'feature': 'Transition',
  'scale': 0.01,
  'sos_value': 0.5866553978765654,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 3.0464971270421334e-11,
  'type': 'sos'},
 {'description': 'TM_NoJumpToParent-1-obj0',
  'eq_sumOfAbs': 3.225977599668334e-06,
  'scale': 100.0,
  'type': 'eq'},
 {'description': 'TM_NoJumpToParent-1-obj0',
  'eq_sumOfAbs': 0.0,
  'scale': 100.0,
  'type': 'eq'},
 {'eq_sumOfAbs': 0.00011630278764720309,
  'feature': 'dist',
  'o1': 'endeff',
  'o2': 'obj0',
  'scale': 100.0,
  'type': 'eq'},
 {'feature': 'above',
  'o1': 'obj0',
  'o2': 'bottom',
  'scale': 10.0,
  'type': 'ineq'},
 {'eq_sumOfAbs': 2.0816681711721685e-17,
  'feature': 'ProxyCost',
  'scale': 10.0,
  'type': 'eq'}]

In [7]:
komo = lgp.getKOMO(0, BT.seqPath)
joints = K.getJointNames()




In [9]:
T = komo.getT()
print(T)

25


In [10]:
import numpy as np
waypoints = []
for i in range(-1, T):
    X = komo.getConfiguration(i)
    K.setFrameState(X)
    joints = K.getJointState()
    if (joints[7] < 0.05):
        joints[7] = 0.00
    else:
        joints[7] = 0.07
    waypoints = np.append(waypoints, np.array([joints]))
wp = np.array(waypoints).reshape(T+1,8)
print(wp)
print(wp[:,-1])
print(wp[:,0:7])
np.save("wp.npy", wp)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.75001000e+00
   0.00000000e+00  1.00001000e+00  0.00000000e+00  7.00000000e-02]
 [ 4.00329884e-03  2.84557240e-02 -1.23907446e-03 -1.74921590e+00
  -1.26923306e-02  9.80901996e-01  4.38578097e-03  7.00000000e-02]
 [ 1.10355521e-02  8.14914612e-02 -4.16472054e-03 -1.75180184e+00
  -3.48400466e-02  9.35610221e-01  1.26862257e-02  7.00000000e-02]
 [ 2.01345929e-02  1.55161708e-01 -9.16599288e-03 -1.76152243e+00
  -6.32807145e-02  8.57820836e-01  2.43715194e-02  7.00000000e-02]
 [ 3.02945610e-02  2.45151403e-01 -1.64549354e-02 -1.78121858e+00
  -9.49040742e-02  7.43389727e-01  3.86941984e-02  7.00000000e-02]
 [ 4.04812888e-02  3.46461986e-01 -2.59631899e-02 -1.81222661e+00
  -1.26725231e-01  5.92145609e-01  5.46037849e-02  7.00000000e-02]
 [ 4.96755866e-02  4.53082471e-01 -3.72143692e-02 -1.85380917e+00
  -1.56001263e-01  4.09771069e-01  7.07171485e-02  7.00000000e-02]
 [ 5.69638913e-02  5.57731441e-01 -4.92221435e-02 -1.90274208e+00
  -

In [ ]:
komo.activateCollisionPairs([('obj0', 'bottom')])
komo.optimize()

In [11]:
K.getJointState()

array([-0.00417812,  0.44569367,  0.00230998, -1.47846373,  0.02704303,
        0.52275213,  0.0055884 ,  0.04518039])

In [10]:
komo.displayTrajectory()

RuntimeError: array.tpp:operator():830(-2) CHECK failed: 'nd==3 && (uint)i<d0 && (uint)j<d1 && (uint)k<d2 && !isSparseMatrix(*this)' 3D range error (3=3, 805<806, 0<0, 0<3)

In [ ]:
lgp.stop()

In [9]:
K = 0

In [10]:
V = 0